## Imports and Configuration

In [1]:
import os
import datetime
import wandb

import torch

import pytorch_lightning as pl
from pytorch_lightning import Trainer
from pytorch_lightning.loggers import WandbLogger
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.utilities.model_summary import ModelSummary

from config.load_configuration import load_configuration
from data.datamodule import ECG_DataModule
from model.model import UNET_1D

#### Loading configuration

This notebook loads configuration settings using the `load_configuration` function from the `config.load_configuration` module. The configuration is stored in the `config` variable.

In [2]:
config = load_configuration()

PC Name: DESKTOP-LUKAS
Loaded configuration from config/config_lukas.yaml


#### Logging in to Weights & Biases (wandb)

Before starting any experiment tracking, ensure you are logged in to your Weights & Biases (wandb) account. This enables automatic logging of metrics, model checkpoints, and experiment configurations. The following code logs you in to wandb:

```python
wandb.login()
```
If you are running this for the first time, you may be prompted to enter your API key.

In [3]:
wandb.login()

wandb: Currently logged in as: lukas-pelz (HKA-EKG-Signalverarbeitung) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

#### Setting Seeds for Reproducibility

To ensure comparable and reproducible results, we set the random seed using the `seed_everything` function from PyTorch Lightning. This helps in achieving consistent behavior across multiple runs of the notebook.

In [4]:
pl.seed_everything(config['seed'])
os.environ["TF_ENABLE_ONEDNN_OPTS"] = "0"   # disable oneDNN optimizations for reproducibility
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

Seed set to 42


#### Checking for GPU Devices

In this step, we check for the availability of GPU devices and print the device currently being used by PyTorch. This ensures that the computations are performed on the most efficient hardware available.

In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("="*40)
print(f"Torch Version      : {torch.__version__}")
print(f"Selected Device    : {device}")
if device.type == 'cuda':
    print(f"CUDA Version       : {torch.version.cuda}")
    print(f"Device Name        : {torch.cuda.get_device_name(0)}")
    allocated = torch.cuda.memory_allocated(0) / 1024**3
    reserved = torch.cuda.memory_reserved(0) / 1024**3
    print(f"Memory Usage       : Allocated: {allocated:.2f} GB | Reserved: {reserved:.2f} GB")
    torch.set_float32_matmul_precision('high')
else:
    print("CUDA not available, using CPU.")
print("="*40)

Torch Version      : 2.7.0+cu128
Selected Device    : cuda
CUDA Version       : 12.8
Device Name        : NVIDIA GeForce RTX 5060 Ti
Memory Usage       : Allocated: 0.00 GB | Reserved: 0.00 GB


#### Initializing the Data Module

The `ECG_DataModule` is initialized using the data path, batch size, and feature list from the configuration. This prepares the data for training and validation.

In [6]:
config['data_cols'] = ['II', 'V2', 'V3', 'V4', 'V5', 'V6']

dm = ECG_DataModule(
    data_dir=config['path_to_data'],
    batch_size=config['batch_size'],
    # num_workers=0,
    # persistent_workers=False,
    num_workers=config['num_workers'],
    persistent_workers=config['persistent_workers'],
    feature_list=config['feature_list'],
    data_cols=config['data_cols']
)
dm.setup()
dm.train_dataset.__getitem__(0)  # Warm up dataset (for reproducibility when using num_workers > 0)
print(dm.train_dataset.__getitem__(0))

(tensor([[ 0.0072, -0.0031, -0.0130,  ...,  0.0081,  0.0161,  0.0265],
        [ 0.0135,  0.0051, -0.0003,  ...,  0.0912,  0.1072,  0.1277],
        [ 0.0084,  0.0054,  0.0048,  ...,  0.0759,  0.1139,  0.1587],
        [ 0.0127,  0.0120,  0.0116,  ...,  0.0546,  0.0864,  0.1246],
        [ 0.0275,  0.0271,  0.0261,  ...,  0.0450,  0.0692,  0.0969],
        [ 0.0121,  0.0093,  0.0069,  ...,  0.0095,  0.0300,  0.0530]]), tensor([[1., 1., 1.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 1., 1., 1.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]]))


#### Creating the Model

In this step, we will define the model architecture and print its summary using the `ModelSummary` utility from PyTorch Lightning. This provides an overview of the model's layers, parameters, and structure.

In [7]:
model = UNET_1D(
    in_channels=len(config['data_cols']), 
    layer_n=512, 
    out_channels=len(config['feature_list']), 
    kernel_size=5
)
print(ModelSummary(model, max_depth=-1))  
print(type(model).__name__)

    | Name                                            | Type                  | Params | Mode  | In sizes      | Out sizes    
------------------------------------------------------------------------------------------------------------------------------------
0   | criterion                                       | BCEWithLogitsLoss     | 0      | train | ?             | ?            
1   | train_jaccard                                   | BinaryJaccardIndex    | 0      | train | ?             | ?            
2   | val_jaccard                                     | BinaryJaccardIndex    | 0      | train | ?             | ?            
3   | test_jaccard                                    | BinaryJaccardIndex    | 0      | train | ?             | ?            
4   | multi_tolerance_metrics                         | MultiToleranceWrapper | 0      | train | ?             | ?            
5   | multi_tolerance_metrics.metrics                 | ModuleDict            | 0      | train | ?       

In [8]:
current_time = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M")
print(f"Current Time       : {current_time}")

# Initialize wandb logger (https://wandb.ai/HKA-EKG-Signalverarbeitung)
wandb_logger = WandbLogger(
    project=config['wandb_project_name'],
    name=f"{config['wandb_experiment_name']}_{type(model).__name__}_{current_time}",
    config={
        'model': type(model).__name__,
        'dataset': type(dm).__name__,
        'batch_size': config['batch_size'],
        'max_epochs': config['max_epochs'],
        'learning_rate': config['learning_rate']
    }
)

# Initialize Trainer with wandb logger, using early stopping callback (https://lightning.ai/docs/pytorch/stable/common/early_stopping.html)
trainer = Trainer(
    max_epochs=config['max_epochs'], 
    default_root_dir='model/checkpoint/', #data_directory, 
    accelerator="auto", 
    devices="auto", 
    strategy="auto",
    callbacks=[EarlyStopping(monitor='val_loss', patience=5, mode='min')], 
    logger=wandb_logger)

trainer.fit(model=model, datamodule=dm)

# Finish wandb
wandb.finish()

# Create a filename with date identifier
model_filename = f"{config['wandb_experiment_name']}_{type(model).__name__}_{current_time}.ckpt"
model.model_name = f"{config['wandb_experiment_name']}_{type(model).__name__}_{current_time}"

# Save the model's state_dict to the path specified in config
save_path = os.path.join(config['path_to_models'], model_filename)
trainer.save_checkpoint(save_path)
print(f"Model checkpoint saved as {save_path}")

Using default `ModelCheckpoint`. Consider installing `litmodels` package to enable `LitModelCheckpoint` for automatic upload to the Lightning model registry.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


Current Time       : 2025-09-16_10-43


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

   | Name                    | Type                  | Params | Mode  | In sizes      | Out sizes    
-----------------------------------------------------------------------------------------------------------
0  | criterion               | BCEWithLogitsLoss     | 0      | train | ?             | ?            
1  | train_jaccard           | BinaryJaccardIndex    | 0      | train | ?             | ?            
2  | val_jaccard             | BinaryJaccardIndex    | 0      | train | ?             | ?            
3  | test_jaccard            | BinaryJaccardIndex    | 0      | train | ?             | ?            
4  | multi_tolerance_metrics | MultiToleranceWrapper | 0      | train | ?             | ?            
5  | AvgPool1D1              | AvgPool1d             | 0      | train | [1, 64, 512]  | [1, 64, 256] 
6  | AvgPool1D2              | AvgPool1d             | 0      | train | [1, 128, 256] | [1, 128, 128]
7  | AvgPool1D3              | Av

Epoch 49: 100%|██████████| 477/477 [00:32<00:00, 14.70it/s, v_num=t5j8, train_loss_step=0.0627, val_loss_step=0.0535, val_loss_epoch=0.0642, train_loss_epoch=0.0627]

`Trainer.fit` stopped: `max_epochs=50` reached.


Epoch 49: 100%|██████████| 477/477 [00:33<00:00, 14.34it/s, v_num=t5j8, train_loss_step=0.0627, val_loss_step=0.0535, val_loss_epoch=0.0642, train_loss_epoch=0.0627]


wandb: ERROR The nbformat package was not found. It is required to save notebook history.


epoch,▁▁▁▁▁▂▂▂▂▂▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇▇███
train_loss_epoch,█▅▄▃▃▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_loss_step,█▇▆▅▄▄▃▂▂▂▂▂▂▂▂▂▁▂▂▁▁▁▂▁▁▁▁▁▁▂▁▁▁▁▁▁▁▁▁▁
trainer/global_step,▁▁▁▁▁▁▁▁▁▂▂▂▂▂▂▄▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂█▂▂
val_loss_epoch,█▆▅▃▃▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_loss_step,██▅▄▄▂▁▁▂▁▁▁▁▁▂▁▂▁▁▁▂▁▁▁▁▁▂▁▁▁▁▁▁▁▁▂▁▁▁▁
epoch,49
train_loss_epoch,0.06268
train_loss_step,0.06269
trainer/global_step,23849
val_loss_epoch,0.06417


Model checkpoint saved as C:\Users\lukas\Documents\HKA_DEV\HKA_EKG_Signalverarbeitung_Data\models\manual_training_1_lead_UNET_1D_2025-09-16_10-43.ckpt


In [6]:
import optuna
from training.hyperparameter_optimization import OptunaTrainer

def objective(trial):
    model = UNET_1D
    config["sweep_id"] = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M")
    config["dataset_name"] = "ECG_TEST_DATASET"
    config["model_name"] = type(model).__name__
    trainer = OptunaTrainer(
        model=model,
        config=config
    )

    # Run training and return validation loss
    val_loss = trainer.run_training(trial)

    # Run testing to save test results / metrics of trained model
    trainer.run_test()

    return val_loss

# Optuna Hyperparameter Study
study = optuna.create_study(direction="minimize", study_name=f"Optuna_HPO_{datetime.datetime.now().strftime('%Y-%m-%d_%H-%M')}")

# Reduce output clutter by setting verbosity to WARNING
optuna.logging.set_verbosity(optuna.logging.WARNING)

# Start optimization
study.optimize(objective, n_trials=config['number_of_trials'], gc_after_trial=True, show_progress_bar=True)

# Best result
print("Best trial: ", study.best_trial)
print("Best value (loss): ", study.best_value)

c:\Users\lukas\anaconda3\envs\HKA_EKG_Signalverarbeitung\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[I 2025-09-16 12:48:46,847] A new study created in memory with name: Optuna_HPO_2025-09-16_12-48
  0%|          | 0/5 [00:00<?, ?it/s]c:\Users\lukas\anaconda3\envs\HKA_EKG_Signalverarbeitung\Lib\site-packages\optuna\distributions.py:699: UserWarning: The distribution is specified by [3, 5] and step=10, but the range is not divisible by `step`. It will be replaced by [3, 3].
  warnings.warn(
Using 16bit Automatic Mixed Precision (AMP)
Using default `ModelCheckpoint`. Consider installing `litmodels` package to enable `LitModelCheckpoint` for automatic upload to the Lightning model registry.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
Using de

['II', 'V2', 'V3', 'V4', 'V5', 'V6']
II-V2-V3-V4-V5-V6
Error occurred during training: 1 validation error for Settings
run_name
  Input should be a valid string [type=string_type, input_value=('type_bs16_lr4.5e-04_wd2...6', '_2025-09-16_12-48'), input_type=tuple]
    For further information visit https://errors.pydantic.dev/2.11/v/string_type
('type_bs16_lr4.5e-04_wd2.7e-03_optAdam_schCosineAnnealingLR_acc1', '_leadsII-V2-V3-V4-V5-V6', '_2025-09-16_12-48')
[W 2025-09-16 12:48:46,939] Trial 0 failed with parameters: {'batch_size': 16, 'max_epochs': 3, 'accumulate_grad_batches': 1, 'precision': '16-mixed', 'optimizer': 'Adam', 'learning_rate': 0.0004501038542985279, 'weight_decay': 0.0027286606090991842, 'scheduler': 'CosineAnnealingLR', 'I': 1, 'II': 0, 'III': 0, 'aVR': 0, 'aVL': 1, 'aVF': 0, 'V1': 0, 'V2': 0, 'V3': 0, 'V4': 0, 'V5': 0, 'V6': 1} because of the following error: TypeError("join() argument must be str, bytes, or os.PathLike object, not 'tuple'").
Traceback (most recent cal

  0%|          | 0/5 [00:00<?, ?it/s]


TypeError: join() argument must be str, bytes, or os.PathLike object, not 'tuple'